![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/30.Clinical_Text_Classification_with_Spark_NLP.ipynb)

# Colab Setup

In [ ]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing spark-nlp for GPU
!wget https://setup.johnsnowlabs.com/colab.sh -O - | bash /dev/stdin -p 3.2.3 -s $PUBLIC_VERSION -g

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [ ]:
!pip install -q tensorflow==2.11.0
!pip install -q tensorflow-addons

**PLEASE RE-START RUNTIME AND CONTINUE**

In [1]:
import json
import os

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [2]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline,PipelineModel

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"24G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params, gpu=True)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.3.0
Spark NLP_JSL Version : 4.3.0


# Classifiers

The below classifiers will be used in this notebook.ClassifierDL, MultiClassifierDL, and GenericClassifier will be trained using healthcare_100d, embeddingd_clinical, and bert sentence embeddings(sbiobert_base_cased_mli). DocumentLogRegClassifier accepts tokens, so sentence embeddings are not utilized during DocumentLogRegClassifier training.

## ClassifierDL

ClassifierDL is a generic Multi-class Text Classification annotator. ClassifierDL uses the state-of-the-art Universal Sentence Encoder as an input for text classifications. The ClassifierDL annotator uses a deep learning model (DNNs) built inside TensorFlow and supports up to 100 classes. For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#classifierdl).


##  MultiClassifierDL

MultiClassifierDL is a Multi-label Text Classification annotator.MultiClassifierDL uses a Bidirectional GRU with a convolutional model built inside TensorFlow and supports up to 100 classes.  For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#multiclassifierdl).


## GenericClassifier

GenericClassifier is a TensorFlow model for the generic classification of feature vectors in Healthcare  Lİbrary. It takes FEATURE_VECTOR annotations from FeaturesAssembler as input, classifies them, and outputs CATEGORY annotations. Please see [the link](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#genericclassifier) for more information.

## GenericLogRegClassifierApproach

`GenericLogRegClassifierApproach` is a derivative of GenericClassifier which implements a multinomial *Logistic Regression*. This is a single layer neural network with the logistic function at the output. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores varying between 0 and 1. Training data requires "text" and their "label" columns only and the trained model will be a `GenericLogRegClassifierModel()`.


## GenericSVMClassifierApproach

`GenericSVMClassifierApproach` is a derivative of GenericClassifier which implements *SVM (Support Vector Machine)* classification. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores. The scores are standardized using the logistic function so that they vary between 0 and 1. Taining data requires "text" and their "label" columns only and the trained model will be a `GenericSVMClassifierModel()`

## DocumentLogRegClassifier

DocumentLogRegClassifier is a model to classify documents with a Logarithmic Regression algorithm in Healthcare  Library. Training data requires columns for text and labels. The result is a trained DocumentLogRegClassifierModel. you can get more info [here](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#documentlogregclassifier). 


# ADE Dataset

### Data Preprocessing

In [3]:
#downloading sample datasets
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

**ADE-Negative Dataset**

In [4]:
df_neg= pd.read_csv("ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg.head()

,col1
0,6460590 NEG Clioquinol intoxication occurring ...
1,"8600337 NEG ""Retinoic acid syndrome"" was preve..."
2,8402502 NEG BACKGROUND: External beam radiatio...
3,"8700794 NEG Although the enuresis ceased, she ..."
4,17662448 NEG A 42-year-old woman had uneventfu...


In [5]:
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]
df_neg.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


**ADE Positive Dataset**

In [6]:
df_pos= pd.read_csv("DRUG-AE.rel", header=None, delimiter="|")
df_pos.head()

,0,1,2,3,4,5,6,7
0,10030778,Intravenous azithromycin-induced ototoxicity.,ototoxicity,43,54,azithromycin,22,34
1,10048291,"Immobilization, while Paget's bone disease was...",increased calcium-release,960,985,dihydrotachysterol,908,926
2,10048291,Unaccountable severe hypercalcemia in a patien...,hypercalcemia,31,44,dihydrotachysterol,94,112
3,10082597,METHODS: We report two cases of pseudoporphyri...,pseudoporphyria,620,635,naproxen,646,654
4,10082597,METHODS: We report two cases of pseudoporphyri...,pseudoporphyria,620,635,oxaprozin,659,668


In [7]:
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]
df_pos.head()

,text,category
0,Intravenous azithromycin-induced ototoxicity.,pos
1,"Immobilization, while Paget's bone disease was...",pos
2,Unaccountable severe hypercalcemia in a patien...,pos
3,METHODS: We report two cases of pseudoporphyri...,pos
4,METHODS: We report two cases of pseudoporphyri...,pos


**Merging Positive and Negative dataset**

In [8]:
ade_df= pd.concat([df_neg, df_pos])
ade_df.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


In [9]:
ade_df["category"].value_counts()

neg    16695
pos     6821
Name: category, dtype: int64

In [10]:
ade_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23516 entries, 0 to 6820
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      23516 non-null  object
 1   category  23516 non-null  object
dtypes: object(2)
memory usage: 551.2+ KB


We take 30% of the data to make a faster run. You can use all data for better scores.

In [11]:
spark_df = spark.createDataFrame(ade_df).sample(0.3, 3) # limit the data

trainingData, testData = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 5531
Test Dataset Count: 1427


In [12]:
spark_df.groupBy("category").count().show()

+--------+-----+
|category|count|
+--------+-----+
|     neg| 4932|
|     pos| 2026|
+--------+-----+



In [13]:
spark_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)



In [14]:
spark_df.head(3)

[Row(text=' Clioquinol intoxication occurring in the treatment of acrodermatitis enteropathica with reference to SMON outside of Japan.', category='neg'),
 Row(text=' A 42-year-old woman had uneventful bilateral laser-assisted subepithelial keratectomy (LASEK) to correct myopia.', category='neg'),
 Row(text=' A 16-year-old girl with erosive, polyarticular JRA showed no detectable change in her articular disease following nine exchanges.', category='neg')]

## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



Now we will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [15]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [16]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| "Syndrome malin"-like symptoms probably due to interaction between neurolept...|     neg|[{sentence_embeddings, 0, 109,  "Syndrome malin"-like symptoms probably due t...|
| 'Bail-out' bivalirudin use in patients with thrombotic complications unrespo...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin use in patients with t...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [17]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| 1 previously untreated patient with plasma-cell leukaemia and 8 patients wit...|     neg|[{sentence_embeddings, 0, 166,  1 previously untreated patient with plasma-ce...|
| A 15-year-old boy had temporary hypertropia, supraduction deficit, ipsilater...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had temporary hypertropia, ...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [18]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [19]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [3]:
log_folder="ADE_logs_healthcare_100d"
!mkdir -p $log_folder

### ClassifierDLApproach

In [21]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction")\
        .setLabelColumn("category")\
        .setBatchSize(16)\
        .setMaxEpochs(30)\
        .setLr(0.002)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)\
        # .setValidationSplit(0.1)

classifier_dl_pipeline = Pipeline(
    stages = [
        classifier_dl
    ])

In [22]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [4]:
!cat $log_folder/ClassifierDLApproach_*

Training started - epochs: 30 - learning_rate: 0.002 - batch_size: 16 - training_examples: 5531 - classes: 2
Epoch 0/30 - 1.81s - loss: 196.50443 - acc: 0.75709814 - batches: 346
Epoch 1/30 - 0.72s - loss: 178.57925 - acc: 0.8049242 - batches: 346
Epoch 2/30 - 0.68s - loss: 171.11334 - acc: 0.8276515 - batches: 346
Epoch 3/30 - 0.67s - loss: 167.70439 - acc: 0.83852106 - batches: 346
Epoch 4/30 - 0.70s - loss: 165.45164 - acc: 0.8476778 - batches: 346
Epoch 5/30 - 0.68s - loss: 163.55579 - acc: 0.8542819 - batches: 346
Epoch 6/30 - 0.68s - loss: 162.0681 - acc: 0.85645586 - batches: 346
Epoch 7/30 - 0.71s - loss: 160.86972 - acc: 0.8609025 - batches: 346
Epoch 8/30 - 0.70s - loss: 159.76674 - acc: 0.86633724 - batches: 346
Epoch 9/30 - 0.73s - loss: 158.82518 - acc: 0.86869234 - batches: 346
Epoch 10/30 - 0.71s - loss: 158.01875 - acc: 0.8714097 - batches: 346
Epoch 11/30 - 0.79s - loss: 157.29413 - acc: 0.87394595 - batches: 346
Epoch 12/30 - 0.73s - loss: 156.6187 - acc: 0.8755764 - 

In [24]:
preds = clfDL_model_hc100.transform(testData_with_embeddings)

In [25]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    

In [26]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.88      0.88      0.88       982
         pos       0.73      0.72      0.73       445

    accuracy                           0.83      1427
   macro avg       0.80      0.80      0.80      1427
weighted avg       0.83      0.83      0.83      1427



### MultiClassifierDL

We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes. It is designed for multi-label classification purposes. Here we will use MultiClassifierDL as a binary classifier. 

In [27]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [28]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(16)\
  .setMaxEpochs(15)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

In [29]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [5]:
!cat $log_folder/MultiClassifierDLApproach_*

Training started - epochs: 15 - learning_rate: 0.009 - batch_size: 16 - training_examples: 5531 - classes: 2
Epoch 0/15 - 14.08s - loss: 0.51671696 - acc: 0.7443017 - batches: 346
Epoch 1/15 - 4.39s - loss: 0.4562747 - acc: 0.77980894 - batches: 346
Epoch 2/15 - 4.66s - loss: 0.42084727 - acc: 0.8010952 - batches: 346
Epoch 3/15 - 3.15s - loss: 0.40065733 - acc: 0.81277996 - batches: 346
Epoch 4/15 - 2.89s - loss: 0.38565522 - acc: 0.8242836 - batches: 346
Epoch 5/15 - 3.10s - loss: 0.37033468 - acc: 0.8311594 - batches: 346
Epoch 6/15 - 3.33s - loss: 0.3562811 - acc: 0.83894926 - batches: 346
Epoch 7/15 - 2.94s - loss: 0.34204647 - acc: 0.84257245 - batches: 346
Epoch 8/15 - 2.89s - loss: 0.32721782 - acc: 0.85018116 - batches: 346
Epoch 9/15 - 2.87s - loss: 0.31454432 - acc: 0.8586051 - batches: 346
Epoch 10/15 - 3.03s - loss: 0.2998448 - acc: 0.8674819 - batches: 346
Epoch 11/15 - 3.46s - loss: 0.28469044 - acc: 0.87527174 - batches: 346
Epoch 12/15 - 2.77s - loss: 0.26784799 - acc:

In [31]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings)

In [32]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category_array, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [33]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']           1145
['pos']            281
['neg', 'pos']       1
Name: result, dtype: int64

In [34]:
preds_df[preds_df.result.apply(len)==2]

,category,text,result,metadata
225,neg,This report describes a male college student ...,"[neg, pos]","[{'sentence': '0', 'neg': '0.5002521', 'pos': ..."


In [35]:
preds_df[preds_df.result.apply(len)==0]

,category,text,result,metadata


MultiClassifierDL is a multi-label classifier, so some predictions may include both labels or none of the labels. That can be controlled a bit with `.setThreshold()` parameter during training. For now we will keep not keep zero label predictions and get the highest score as prediction.

In [36]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

neg    1145
pos     282
Name: result, dtype: int64

In [37]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.80      0.94      0.86       982
         pos       0.78      0.49      0.60       445

    accuracy                           0.80      1427
   macro avg       0.79      0.71      0.73      1427
weighted avg       0.79      0.80      0.78      1427



### Generic Classifier

In [38]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

GenericClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [39]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")
      
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [40]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}


Instructions for updating:
Colocations handled automatically by placer.


generic_classifier graph exported to gc_graph/gcf_graph_100d.pb


In [6]:
!cat $log_folder/GenericClassifierApproach_*

Training 25 epochs
Epoch 1/25	0.43s	Loss: 11.13918	ACC: 0.6618989
Epoch 2/25	0.13s	Loss: 9.798145	ACC: 0.7244449
Epoch 3/25	0.13s	Loss: 9.700033	ACC: 0.71773726
Epoch 4/25	0.12s	Loss: 9.878727	ACC: 0.7129498
Epoch 5/25	0.12s	Loss: 9.169177	ACC: 0.72923905
Epoch 6/25	0.16s	Loss: 8.999225	ACC: 0.7426807
Epoch 7/25	0.15s	Loss: 8.661567	ACC: 0.7545836
Epoch 8/25	0.16s	Loss: 8.716748	ACC: 0.7564907
Epoch 9/25	0.12s	Loss: 8.892795	ACC: 0.7467317
Epoch 10/25	0.12s	Loss: 8.50721	ACC: 0.7592922
Epoch 11/25	0.12s	Loss: 8.547359	ACC: 0.76918274
Epoch 12/25	0.12s	Loss: 8.387141	ACC: 0.77105695
Epoch 13/25	0.12s	Loss: 8.277868	ACC: 0.76980746
Epoch 14/25	0.13s	Loss: 8.375022	ACC: 0.7652765
Epoch 15/25	0.12s	Loss: 8.183036	ACC: 0.77522624
Epoch 16/25	0.12s	Loss: 8.118612	ACC: 0.77611405
Epoch 17/25	0.12s	Loss: 8.074482	ACC: 0.77704126
Epoch 18/25	0.11s	Loss: 8.172344	ACC: 0.7773964
Epoch 19/25	0.11s	Loss: 8.042446	ACC: 0.78157884
Epoch 20/25	0.11s	Loss: 7.932385	ACC: 0.78547853
Epoch 21/25	0.11s	Los

In [42]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [43]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [44]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.92      0.76      0.83       982
         pos       0.61      0.85      0.71       445

    accuracy                           0.78      1427
   macro avg       0.76      0.80      0.77      1427
weighted avg       0.82      0.78      0.79      1427



### GenericLogRegClassifier

In [45]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [46]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [47]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [7]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 25 epochs
Epoch 1/25	0.20s	Loss: 26.689623	ACC: 0.7111282
Epoch 2/25	0.08s	Loss: 25.617167	ACC: 0.71246314
Epoch 3/25	0.08s	Loss: 24.814148	ACC: 0.7154751
Epoch 4/25	0.08s	Loss: 24.290005	ACC: 0.72040063
Epoch 5/25	0.08s	Loss: 24.036684	ACC: 0.71752685
Epoch 6/25	0.06s	Loss: 23.708939	ACC: 0.72972566
Epoch 7/25	0.06s	Loss: 23.492151	ACC: 0.7272464
Epoch 8/25	0.06s	Loss: 23.16531	ACC: 0.73572314
Epoch 9/25	0.06s	Loss: 22.959356	ACC: 0.7367819
Epoch 10/25	0.06s	Loss: 23.013765	ACC: 0.7340791
Epoch 11/25	0.06s	Loss: 22.64423	ACC: 0.741622
Epoch 12/25	0.06s	Loss: 22.757103	ACC: 0.7398464
Epoch 13/25	0.07s	Loss: 22.493414	ACC: 0.74215466
Epoch 14/25	0.07s	Loss: 22.638077	ACC: 0.74486405
Epoch 15/25	0.06s	Loss: 22.581057	ACC: 0.7410038
Epoch 16/25	0.06s	Loss: 22.362005	ACC: 0.7445944
Epoch 17/25	0.06s	Loss: 22.519594	ACC: 0.7456203
Epoch 18/25	0.06s	Loss: 22.322456	ACC: 0.7465015
Epoch 19/25	0.06s	Loss: 22.319033	ACC: 0.7507234
Epoch 20/25	0.06s	Loss: 22.139994	ACC: 0.74743533
Epoch

In [49]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [50]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [51]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.74      0.95      0.83       982
         pos       0.72      0.27      0.39       445

    accuracy                           0.74      1427
   macro avg       0.73      0.61      0.61      1427
weighted avg       0.74      0.74      0.70      1427



### GenericSVMClassifier

In [52]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [53]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [54]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [8]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.17s	Loss: 37.366035	ACC: 0.6397964
Epoch 2/25	0.08s	Loss: 26.486511	ACC: 0.7139691
Epoch 3/25	0.09s	Loss: 25.912964	ACC: 0.7150344
Epoch 4/25	0.09s	Loss: 25.395231	ACC: 0.71587616
Epoch 5/25	0.09s	Loss: 25.143248	ACC: 0.71618533
Epoch 6/25	0.09s	Loss: 24.978365	ACC: 0.7186711
Epoch 7/25	0.10s	Loss: 25.133312	ACC: 0.7183225
Epoch 8/25	0.08s	Loss: 24.935448	ACC: 0.71898675
Epoch 9/25	0.09s	Loss: 24.904512	ACC: 0.72102535
Epoch 10/25	0.09s	Loss: 24.807228	ACC: 0.7224918
Epoch 11/25	0.08s	Loss: 24.63386	ACC: 0.7267006
Epoch 12/25	0.08s	Loss: 24.914183	ACC: 0.721078
Epoch 13/25	0.08s	Loss: 24.535143	ACC: 0.7288773
Epoch 14/25	0.09s	Loss: 24.594475	ACC: 0.7267927
Epoch 15/25	0.08s	Loss: 24.52173	ACC: 0.7315012
Epoch 16/25	0.08s	Loss: 24.566565	ACC: 0.73500633
Epoch 17/25	0.08s	Loss: 24.58638	ACC: 0.73274416
Epoch 18/25	0.08s	Loss: 24.453281	ACC: 0.7388205
Epoch 19/25	0.09s	Loss: 24.653605	ACC: 0.7315933
Epoch 20/25	0.08s	Loss: 24.356182	ACC: 0.73957676
Epoch 2

In [56]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [57]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [58]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.72      0.98      0.83       982
         pos       0.77      0.16      0.26       445

    accuracy                           0.72      1427
   macro avg       0.75      0.57      0.55      1427
weighted avg       0.74      0.72      0.65      1427



## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [59]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [60]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| "Syndrome malin"-like symptoms probably due to interaction between neurolept...|     neg|[{sentence_embeddings, 0, 109,  "Syndrome malin"-like symptoms probably due t...|
| 'Bail-out' bivalirudin use in patients with thrombotic complications unrespo...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin use in patients with t...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [61]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| 1 previously untreated patient with plasma-cell leukaemia and 8 patients wit...|     neg|[{sentence_embeddings, 0, 166,  1 previously untreated patient with plasma-ce...|
| A 15-year-old boy had temporary hypertropia, supraduction deficit, ipsilater...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had temporary hypertropia, ...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [62]:
log_folder="ADE_logs_healthcare_200d"

### ClassifierDLApproach

In [63]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(16)\
        .setMaxEpochs(30)\
        .setLr(0.001)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(
    stages = [
        classifier_dl
    ])

In [64]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [65]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.86      0.92      0.88       982
         pos       0.78      0.66      0.71       445

    accuracy                           0.84      1427
   macro avg       0.82      0.79      0.80      1427
weighted avg       0.83      0.84      0.83      1427



### MultiClassifierDL

In [66]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [67]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(32)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

In [68]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [69]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [70]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']    957
['pos']    470
Name: result, dtype: int64

In [71]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

neg    957
pos    470
Name: result, dtype: int64

In [72]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.91      0.89      0.90       982
         pos       0.76      0.80      0.78       445

    accuracy                           0.86      1427
   macro avg       0.84      0.85      0.84      1427
weighted avg       0.86      0.86      0.86      1427



### Generic Classifier

In [73]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [74]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

In [75]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")
      
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(40)\
    .setBatchSize(16)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [76]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb


In [77]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.91      0.84      0.87       982
         pos       0.70      0.82      0.75       445

    accuracy                           0.83      1427
   macro avg       0.80      0.83      0.81      1427
weighted avg       0.85      0.83      0.84      1427



### GenericLogRegClassifier

In [78]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [79]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [80]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [81]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 25 epochs
Epoch 1/25	0.16s	Loss: 26.194351	ACC: 0.71147674
Epoch 2/25	0.06s	Loss: 24.627851	ACC: 0.7151265
Epoch 3/25	0.06s	Loss: 23.502913	ACC: 0.72266936
Epoch 4/25	0.06s	Loss: 22.843657	ACC: 0.7355785
Epoch 5/25	0.06s	Loss: 22.465397	ACC: 0.7412274
Epoch 6/25	0.06s	Loss: 21.985895	ACC: 0.7518348
Epoch 7/25	0.06s	Loss: 21.585283	ACC: 0.7614623
Epoch 8/25	0.06s	Loss: 21.273968	ACC: 0.7592001
Epoch 9/25	0.06s	Loss: 21.320953	ACC: 0.75552404
Epoch 10/25	0.07s	Loss: 21.042255	ACC: 0.7669666
Epoch 11/25	0.06s	Loss: 20.610676	ACC: 0.7710898
Epoch 12/25	0.06s	Loss: 20.555138	ACC: 0.7742398
Epoch 13/25	0.06s	Loss: 20.530922	ACC: 0.7706492
Epoch 14/25	0.06s	Loss: 20.464844	ACC: 0.77087283
Epoch 15/25	0.06s	Loss: 20.532772	ACC: 0.77602196
Epoch 16/25	0.06s	Loss: 20.306566	ACC: 0.7724774
Epoch 17/25	0.06s	Loss: 20.187925	ACC: 0.78178924
Epoch 18/25	0.06s	Loss: 20.065956	ACC: 0.7819668
Epoch 19/25	0.06s	Loss: 20.0021	ACC: 0.7767322
Epoch 20/25	0.06s	Loss: 19.97224	ACC: 0.7769558
Epoch 2

In [82]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [83]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [84]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.79      0.93      0.85       982
         pos       0.74      0.47      0.57       445

    accuracy                           0.78      1427
   macro avg       0.77      0.70      0.71      1427
weighted avg       0.78      0.78      0.77      1427



### GenericSVMClassifier

In [85]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [86]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [87]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [88]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.17s	Loss: 28.306355	ACC: 0.7144557
Epoch 2/25	0.07s	Loss: 25.269861	ACC: 0.71427816
Epoch 3/25	0.07s	Loss: 24.440134	ACC: 0.7192958
Epoch 4/25	0.07s	Loss: 24.030323	ACC: 0.7255103
Epoch 5/25	0.07s	Loss: 23.41503	ACC: 0.7396162
Epoch 6/25	0.07s	Loss: 23.088697	ACC: 0.748448
Epoch 7/25	0.07s	Loss: 22.912378	ACC: 0.75777304
Epoch 8/25	0.07s	Loss: 22.926525	ACC: 0.7664799
Epoch 9/25	0.07s	Loss: 22.586903	ACC: 0.76550007
Epoch 10/25	0.07s	Loss: 22.333496	ACC: 0.77349013
Epoch 11/25	0.07s	Loss: 22.273605	ACC: 0.7722078
Epoch 12/25	0.07s	Loss: 22.246626	ACC: 0.7789549
Epoch 13/25	0.07s	Loss: 22.369463	ACC: 0.78042144
Epoch 14/25	0.07s	Loss: 22.106396	ACC: 0.77624553
Epoch 15/25	0.07s	Loss: 21.948631	ACC: 0.7797112
Epoch 16/25	0.07s	Loss: 21.644228	ACC: 0.7833412
Epoch 17/25	0.07s	Loss: 21.963099	ACC: 0.7784223
Epoch 18/25	0.07s	Loss: 21.934553	ACC: 0.7815262
Epoch 19/25	0.07s	Loss: 21.287962	ACC: 0.790312
Epoch 20/25	0.07s	Loss: 21.574465	ACC: 0.78529435
Epoch 

In [89]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [90]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [91]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.80      0.93      0.86       982
         pos       0.75      0.50      0.60       445

    accuracy                           0.79      1427
   macro avg       0.78      0.71      0.73      1427
weighted avg       0.79      0.79      0.78      1427



## Bert Sentence Embeddings (sbiobert_base_cased_mli)

Now we will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [92]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
bert_sent = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = Pipeline(
    stages = [document_assembler,
              bert_sent])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [93]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| "Syndrome malin"-like symptoms probably due to interaction between neurolept...|     neg|[{sentence_embeddings, 0, 109,  "Syndrome malin"-like symptoms probably due t...|
| 'Bail-out' bivalirudin use in patients with thrombotic complications unrespo...|     neg|[{sentence_embeddings, 0, 150,  'Bail-out' bivalirudin use in patients with t...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [94]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|                                                                            text|category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
| 1 previously untreated patient with plasma-cell leukaemia and 8 patients wit...|     neg|[{sentence_embeddings, 0, 166,  1 previously untreated patient with plasma-ce...|
| A 15-year-old boy had temporary hypertropia, supraduction deficit, ipsilater...|     neg|[{sentence_embeddings, 0, 262,  A 15-year-old boy had temporary hypertropia, ...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------

In [95]:
log_folder="ADE_logs_bert"

### ClassifierDLApproach

In [96]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(2)\
        .setLr(0.001)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

In [97]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [98]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.69      1.00      0.82       982
         pos       0.00      0.00      0.00       445

    accuracy                           0.69      1427
   macro avg       0.34      0.50      0.41      1427
weighted avg       0.47      0.69      0.56      1427



### MultiClassifierDL

In [99]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [100]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(32)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

In [101]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [102]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [103]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']    965
['pos']    462
Name: result, dtype: int64

In [104]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

neg    965
pos    462
Name: result, dtype: int64

In [105]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.91      0.90      0.90       982
         pos       0.78      0.81      0.79       445

    accuracy                           0.87      1427
   macro avg       0.85      0.85      0.85      1427
weighted avg       0.87      0.87      0.87      1427



### Generic Classifier

In [106]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [107]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

In [108]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")
      
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [109]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb


In [110]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.94      0.87      0.90       982
         pos       0.75      0.88      0.81       445

    accuracy                           0.87      1427
   macro avg       0.85      0.87      0.86      1427
weighted avg       0.88      0.87      0.87      1427



### GenericLogRegClassifier

In [111]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [112]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [113]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [114]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 25 epochs
Epoch 1/25	0.18s	Loss: 21.9896	ACC: 0.7652765
Epoch 2/25	0.07s	Loss: 16.533005	ACC: 0.8334583
Epoch 3/25	0.07s	Loss: 15.805766	ACC: 0.8406592
Epoch 4/25	0.07s	Loss: 14.927349	ACC: 0.852766
Epoch 5/25	0.07s	Loss: 14.58918	ACC: 0.85231876
Epoch 6/25	0.07s	Loss: 14.25055	ACC: 0.8595591
Epoch 7/25	0.07s	Loss: 14.322069	ACC: 0.8570273
Epoch 8/25	0.07s	Loss: 13.8823805	ACC: 0.8632418
Epoch 9/25	0.07s	Loss: 13.842393	ACC: 0.8648398
Epoch 10/25	0.07s	Loss: 13.638076	ACC: 0.8640835
Epoch 11/25	0.07s	Loss: 13.571218	ACC: 0.865504
Epoch 12/25	0.08s	Loss: 13.25669	ACC: 0.86758864
Epoch 13/25	0.08s	Loss: 13.736311	ACC: 0.861874
Epoch 14/25	0.08s	Loss: 13.611635	ACC: 0.8626631
Epoch 15/25	0.08s	Loss: 13.468305	ACC: 0.8614202
Epoch 16/25	0.07s	Loss: 13.621185	ACC: 0.8576981
Epoch 17/25	0.07s	Loss: 13.820888	ACC: 0.86297876
Epoch 18/25	0.07s	Loss: 13.579946	ACC: 0.8604009
Epoch 19/25	0.07s	Loss: 13.297374	ACC: 0.8622225
Epoch 20/25	0.07s	Loss: 13.535712	ACC: 0.8667535
Epoch 21/25	0.

In [115]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [116]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [117]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.88      0.94      0.91       982
         pos       0.84      0.72      0.78       445

    accuracy                           0.87      1427
   macro avg       0.86      0.83      0.84      1427
weighted avg       0.87      0.87      0.87      1427



### GenericSVMClassifier

In [118]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [119]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [120]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [121]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.21s	Loss: 25.396908	ACC: 0.76753867
Epoch 2/25	0.08s	Loss: 18.124996	ACC: 0.83168274
Epoch 3/25	0.08s	Loss: 16.69394	ACC: 0.84721565
Epoch 4/25	0.09s	Loss: 17.57085	ACC: 0.83750266
Epoch 5/25	0.09s	Loss: 16.188065	ACC: 0.8548966
Epoch 6/25	0.08s	Loss: 15.183879	ACC: 0.8542719
Epoch 7/25	0.09s	Loss: 15.633756	ACC: 0.8538839
Epoch 8/25	0.08s	Loss: 15.63288	ACC: 0.85538983
Epoch 9/25	0.09s	Loss: 15.137466	ACC: 0.8627091
Epoch 10/25	0.08s	Loss: 15.669785	ACC: 0.86044693
Epoch 11/25	0.08s	Loss: 15.331658	ACC: 0.854364
Epoch 12/25	0.08s	Loss: 15.072625	ACC: 0.8622225
Epoch 13/25	0.09s	Loss: 15.028588	ACC: 0.8606705
Epoch 14/25	0.08s	Loss: 15.170846	ACC: 0.86044693
Epoch 15/25	0.08s	Loss: 15.985527	ACC: 0.8537458
Epoch 16/25	0.09s	Loss: 15.230682	ACC: 0.86337334
Epoch 17/25	0.08s	Loss: 14.991264	ACC: 0.8643532
Epoch 18/25	0.09s	Loss: 15.240396	ACC: 0.85982877
Epoch 19/25	0.09s	Loss: 14.894712	ACC: 0.86648387
Epoch 20/25	0.08s	Loss: 17.250116	ACC: 0.8440723
Epoc

In [122]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [123]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [124]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.87      0.94      0.90       982
         pos       0.84      0.69      0.76       445

    accuracy                           0.86      1427
   macro avg       0.86      0.82      0.83      1427
weighted avg       0.86      0.86      0.86      1427



## DocumentLogRegClassifier

In [125]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = Stemmer() \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("stem")

logreg = DocumentLogRegClassifierApproach()\
    .setInputCols("stem")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(10)\
    .setTol(1e-6)

clf_Pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    normalizer,
    stopwords_cleaner, 
    stemmer, 
    logreg])
doclogreg_model = clf_Pipeline.fit(trainingData)

In [126]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.86      0.86      0.86       982
         pos       0.70      0.69      0.69       445

    accuracy                           0.81      1427
   macro avg       0.78      0.78      0.78      1427
weighted avg       0.81      0.81      0.81      1427



# Mtsamples Dataset

## Load Dataset

In [3]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mtsamples_classifier.csv

In [4]:
spark_df = spark.read.csv("mtsamples_classifier.csv", header = True)

spark_df.show(10,truncate=100)

+----------------+----------------------------------------------------------------------------------------------------+
|        category|                                                                                                text|
+----------------+----------------------------------------------------------------------------------------------------+
|Gastroenterology| PROCEDURES PERFORMED: Colonoscopy. INDICATIONS: Renewed symptoms likely consistent with active f...|
|Gastroenterology| OPERATION 1. Ivor-Lewis esophagogastrectomy. 2. Feeding jejunostomy. 3. Placement of two right-s...|
|Gastroenterology| PREOPERATIVE DIAGNOSES: 1. Gastroesophageal reflux disease. 2. Chronic dyspepsia. POSTOPERATIVE ...|
|Gastroenterology| PROCEDURE: Colonoscopy. PREOPERATIVE DIAGNOSES: Rectal bleeding and perirectal abscess. POSTOPER...|
|Gastroenterology| PREOPERATIVE DIAGNOSIS: Right colon tumor. POSTOPERATIVE DIAGNOSES: 1. Right colon cancer. 2. As...|
|Gastroenterology| PREOPERATIVE DIAGNOSI

In [5]:
spark_df.printSchema()

root
 |-- category: string (nullable = true)
 |-- text: string (nullable = true)



In [6]:
spark_df.count()

638

In [7]:
spark_df.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|  115|
|       Neurology|  143|
|      Orthopedic|  223|
|Gastroenterology|  157|
+----------------+-----+



In [8]:
(trainingData, testData) = spark_df.randomSplit([0.8, 0.2], seed = 42)

In [9]:
trainingData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   95|
|       Neurology|  121|
|      Orthopedic|  188|
|Gastroenterology|  132|
+----------------+-----+



In [10]:
testData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   20|
|       Neurology|   22|
|      Orthopedic|   35|
|Gastroenterology|   25|
+----------------+-----+



## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



We will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [11]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [12]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
|                                                                            text|        category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHARGE DIAGNOSIS: Sympto...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Symptomatic cholelithia...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal reflux disease reflux. ...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hiatal hernia, gastroes...|
+--------------------------------------------------------------------------------+-------------

In [13]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
|                                                                            text|        category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF PRESENT ILLNESS: Ms....|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gastrointestinal bleed....|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomiting. HISTORY OF PRESEN...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphagia and hematemesis w...|
+--------------------------------------------------------------------------------+-------------

In [14]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [15]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
log_folder="Mt_logs_healthcare_100d"
!mkdir -p $log_folder

### ClassifierDLApproach

In [17]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(50)\
        .setLr(0.005)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)\
        # .setValidationSplit(0.1)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

In [18]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [19]:
preds = clfDL_model_hc100.transform(testData_with_embeddings)

In [20]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.63      0.96      0.76        25
       Neurology       0.64      0.73      0.68        22
      Orthopedic       0.82      0.91      0.86        35
         Urology       0.00      0.00      0.00        20

        accuracy                           0.71       102
       macro avg       0.52      0.65      0.58       102
    weighted avg       0.57      0.71      0.63       102



### MultiClassifierDL

In [21]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [22]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(8)\
  .setMaxEpochs(40)\
  .setLr(5e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

In [23]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [24]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings)

In [25]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']                        35
['Urology']                           21
['Gastroenterology']                  19
['Neurology']                         17
[]                                     5
['Neurology', 'Orthopedic']            4
['Orthopedic', 'Gastroenterology']     1
Name: result, dtype: int64

In [26]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

Orthopedic          36
Urology             21
Gastroenterology    20
Neurology           20
                     5
Name: result, dtype: int64

In [27]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

                       0.00      0.00      0.00         0
Gastroenterology       0.95      0.76      0.84        25
       Neurology       0.80      0.73      0.76        22
      Orthopedic       0.89      0.91      0.90        35
         Urology       0.90      0.95      0.93        20

        accuracy                           0.84       102
       macro avg       0.71      0.67      0.69       102
    weighted avg       0.89      0.84      0.86       102



### Generic Classifier

In [28]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

In [29]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")
      
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(50)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(False)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [30]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}


Instructions for updating:
Colocations handled automatically by placer.


generic_classifier graph exported to gc_graph/gcf_graph_100d.pb


In [31]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [32]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.88      0.90        25
       Neurology       0.81      0.77      0.79        22
      Orthopedic       0.89      0.91      0.90        35
         Urology       0.90      0.95      0.93        20

        accuracy                           0.88       102
       macro avg       0.88      0.88      0.88       102
    weighted avg       0.88      0.88      0.88       102



### GenericLogRegClassifier

In [33]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [34]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [35]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [36]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 25 epochs
Epoch 1/25	0.08s	Loss: 3.3014627	ACC: 0.25
Epoch 2/25	0.01s	Loss: 3.105043	ACC: 0.29010418
Epoch 3/25	0.01s	Loss: 2.9731028	ACC: 0.2682292
Epoch 4/25	0.01s	Loss: 2.8684158	ACC: 0.3385417
Epoch 5/25	0.01s	Loss: 2.8149989	ACC: 0.34270832
Epoch 6/25	0.01s	Loss: 2.758082	ACC: 0.36302084
Epoch 7/25	0.01s	Loss: 2.7520964	ACC: 0.334375
Epoch 8/25	0.01s	Loss: 2.7168496	ACC: 0.34791666
Epoch 9/25	0.01s	Loss: 2.6751735	ACC: 0.38333336
Epoch 10/25	0.01s	Loss: 2.6831145	ACC: 0.3546875
Epoch 11/25	0.01s	Loss: 2.6507664	ACC: 0.36822918
Epoch 12/25	0.01s	Loss: 2.6277065	ACC: 0.38645834
Epoch 13/25	0.01s	Loss: 2.6588118	ACC: 0.359375
Epoch 14/25	0.01s	Loss: 2.6552174	ACC: 0.3375
Epoch 15/25	0.01s	Loss: 2.6105545	ACC: 0.3671875
Epoch 16/25	0.01s	Loss: 2.5974798	ACC: 0.40260416
Epoch 17/25	0.01s	Loss: 2.612493	ACC: 0.41614586
Epoch 18/25	0.01s	Loss: 2.6132898	ACC: 0.42083335
Epoch 19/25	0.01s	Loss: 2.5750844	ACC: 0.44791666
Epoch 20/25	0.01s	Loss: 2.5674896	ACC: 0.49375
Epoch 21/25	0.

In [37]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [38]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [39]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.78      0.28      0.41        25
       Neurology       0.00      0.00      0.00        22
      Orthopedic       0.38      1.00      0.55        35
         Urology       0.00      0.00      0.00        20

        accuracy                           0.41       102
       macro avg       0.29      0.32      0.24       102
    weighted avg       0.32      0.41      0.29       102



### GenericSVMClassifier

In [40]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [41]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [42]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [43]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.09s	Loss: 5.7645335	ACC: 0.32708332
Epoch 2/25	0.01s	Loss: 5.4396033	ACC: 0.346875
Epoch 3/25	0.01s	Loss: 5.2420816	ACC: 0.371875
Epoch 4/25	0.01s	Loss: 5.1173368	ACC: 0.4546875
Epoch 5/25	0.01s	Loss: 5.069068	ACC: 0.48333335
Epoch 6/25	0.01s	Loss: 5.0111904	ACC: 0.43333334
Epoch 7/25	0.01s	Loss: 4.908137	ACC: 0.47708336
Epoch 8/25	0.01s	Loss: 4.8826556	ACC: 0.521875
Epoch 9/25	0.01s	Loss: 4.695424	ACC: 0.55
Epoch 10/25	0.01s	Loss: 4.760127	ACC: 0.5442709
Epoch 11/25	0.01s	Loss: 4.655547	ACC: 0.55364585
Epoch 12/25	0.01s	Loss: 4.628164	ACC: 0.55572915
Epoch 13/25	0.01s	Loss: 4.607344	ACC: 0.58489585
Epoch 14/25	0.01s	Loss: 4.4588337	ACC: 0.6197916
Epoch 15/25	0.01s	Loss: 4.4664936	ACC: 0.6010417
Epoch 16/25	0.01s	Loss: 4.4386697	ACC: 0.6067709
Epoch 17/25	0.01s	Loss: 4.391735	ACC: 0.6265625
Epoch 18/25	0.01s	Loss: 4.3258085	ACC: 0.64270836
Epoch 19/25	0.01s	Loss: 4.376319	ACC: 0.65625
Epoch 20/25	0.01s	Loss: 4.2476535	ACC: 0.63489586
Epoch 21/25	0.01s	Lo

In [44]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [45]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [46]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.64      0.84      0.72        25
       Neurology       0.82      0.64      0.72        22
      Orthopedic       0.65      0.94      0.77        35
         Urology       1.00      0.05      0.10        20

        accuracy                           0.68       102
       macro avg       0.78      0.62      0.58       102
    weighted avg       0.75      0.68      0.61       102



## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [47]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [48]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
|                                                                            text|        category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHARGE DIAGNOSIS: Sympto...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Symptomatic cholelithia...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal reflux disease reflux. ...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hiatal hernia, gastroes...|
+--------------------------------------------------------------------------------+-------------

In [49]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
|                                                                            text|        category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF PRESENT ILLNESS: Ms....|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gastrointestinal bleed....|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomiting. HISTORY OF PRESEN...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphagia and hematemesis w...|
+--------------------------------------------------------------------------------+-------------

In [50]:
log_folder="Mt_logs_healthcare_200d"

### ClassifierDLApproach 

In [51]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(50)\
        .setLr(0.005)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

In [52]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [53]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.76      0.76      0.76        25
       Neurology       0.94      0.68      0.79        22
      Orthopedic       0.87      0.97      0.92        35
         Urology       0.64      0.70      0.67        20

        accuracy                           0.80       102
       macro avg       0.80      0.78      0.78       102
    weighted avg       0.81      0.80      0.80       102



### MultiClassifierDL



In [54]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [55]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(16)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.2)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

In [56]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [57]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [58]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']                        32
['Gastroenterology']                  19
['Urology']                           18
['Neurology']                         13
['Neurology', 'Orthopedic']           11
[]                                     4
['Orthopedic', 'Gastroenterology']     3
['Urology', 'Orthopedic']              1
['Urology', 'Gastroenterology']        1
Name: result, dtype: int64

In [59]:
# We will get the highest score label as result. you can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

Orthopedic          38
Gastroenterology    22
Urology             20
Neurology           18
                     4
Name: result, dtype: int64

In [60]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

                       0.00      0.00      0.00         0
Gastroenterology       1.00      0.88      0.94        25
       Neurology       0.89      0.73      0.80        22
      Orthopedic       0.87      0.94      0.90        35
         Urology       0.90      0.90      0.90        20

        accuracy                           0.87       102
       macro avg       0.73      0.69      0.71       102
    weighted avg       0.91      0.87      0.89       102



### Generic Classifier 

In [61]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [62]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

In [63]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")
      
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [64]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb


In [65]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.91      0.84      0.87        25
       Neurology       0.83      0.86      0.84        22
      Orthopedic       0.89      0.91      0.90        35
         Urology       0.85      0.85      0.85        20

        accuracy                           0.87       102
       macro avg       0.87      0.87      0.87       102
    weighted avg       0.87      0.87      0.87       102



### GenericLogRegClassifier

In [66]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [67]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [68]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [69]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 25 epochs
Epoch 1/25	0.11s	Loss: 3.1532466	ACC: 0.2234375
Epoch 2/25	0.01s	Loss: 2.9040172	ACC: 0.27552083
Epoch 3/25	0.01s	Loss: 2.8123925	ACC: 0.31458333
Epoch 4/25	0.01s	Loss: 2.7618628	ACC: 0.34270832
Epoch 5/25	0.01s	Loss: 2.719881	ACC: 0.36614582
Epoch 6/25	0.01s	Loss: 2.7006655	ACC: 0.34947917
Epoch 7/25	0.01s	Loss: 2.6769	ACC: 0.36302084
Epoch 8/25	0.01s	Loss: 2.7031033	ACC: 0.33385417
Epoch 9/25	0.01s	Loss: 2.6678483	ACC: 0.3619792
Epoch 10/25	0.01s	Loss: 2.6511176	ACC: 0.3765625
Epoch 11/25	0.01s	Loss: 2.6291242	ACC: 0.42447916
Epoch 12/25	0.01s	Loss: 2.5808368	ACC: 0.5328125
Epoch 13/25	0.01s	Loss: 2.5439658	ACC: 0.5
Epoch 14/25	0.01s	Loss: 2.5532155	ACC: 0.44479164
Epoch 15/25	0.01s	Loss: 2.5278938	ACC: 0.39375
Epoch 16/25	0.01s	Loss: 2.5228848	ACC: 0.4328125
Epoch 17/25	0.01s	Loss: 2.50916	ACC: 0.47447914
Epoch 18/25	0.01s	Loss: 2.4812713	ACC: 0.50416666
Epoch 19/25	0.01s	Loss: 2.4657931	ACC: 0.53697914
Epoch 20/25	0.01s	Loss: 2.4450657	ACC: 0.5395833
Epoch 21/25	

In [70]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [71]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [72]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.71      0.60      0.65        25
       Neurology       0.77      0.45      0.57        22
      Orthopedic       0.50      0.97      0.66        35
         Urology       0.00      0.00      0.00        20

        accuracy                           0.58       102
       macro avg       0.50      0.51      0.47       102
    weighted avg       0.51      0.58      0.51       102



### GenericSVMClassifier

In [73]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [74]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [75]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [76]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.10s	Loss: 5.4002724	ACC: 0.31875
Epoch 2/25	0.01s	Loss: 5.28392	ACC: 0.36666667
Epoch 3/25	0.01s	Loss: 5.115946	ACC: 0.4484375
Epoch 4/25	0.01s	Loss: 5.026103	ACC: 0.5015625
Epoch 5/25	0.01s	Loss: 5.008495	ACC: 0.484375
Epoch 6/25	0.01s	Loss: 4.8489227	ACC: 0.55364585
Epoch 7/25	0.01s	Loss: 4.7669983	ACC: 0.6010417
Epoch 8/25	0.01s	Loss: 4.774622	ACC: 0.62395835
Epoch 9/25	0.01s	Loss: 4.5643754	ACC: 0.6510416
Epoch 10/25	0.01s	Loss: 4.568773	ACC: 0.6333333
Epoch 11/25	0.01s	Loss: 4.5348177	ACC: 0.6713542
Epoch 12/25	0.01s	Loss: 4.466629	ACC: 0.6666666
Epoch 13/25	0.01s	Loss: 4.28304	ACC: 0.69895834
Epoch 14/25	0.01s	Loss: 4.2987103	ACC: 0.67760414
Epoch 15/25	0.01s	Loss: 4.2730703	ACC: 0.7140625
Epoch 16/25	0.01s	Loss: 4.201017	ACC: 0.6869792
Epoch 17/25	0.01s	Loss: 4.0265555	ACC: 0.6703125
Epoch 18/25	0.01s	Loss: 4.1114507	ACC: 0.68854165
Epoch 19/25	0.01s	Loss: 3.9995584	ACC: 0.6588541
Epoch 20/25	0.01s	Loss: 4.041861	ACC: 0.6848959
Epoch 21/25	0.01s	L

In [77]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [78]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [79]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.64      0.72      0.68        25
       Neurology       0.65      0.77      0.71        22
      Orthopedic       0.69      0.89      0.78        35
         Urology       1.00      0.15      0.26        20

        accuracy                           0.68       102
       macro avg       0.75      0.63      0.61       102
    weighted avg       0.73      0.68      0.64       102



## Bert Sentence Embeddings (sbiobert_base_cased_mli)

We will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [80]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
bert_sent = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = Pipeline(
    stages = [document_assembler,
              bert_sent])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [81]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
|                                                                            text|        category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHARGE DIAGNOSIS: Sympto...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Symptomatic cholelithia...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal reflux disease reflux. ...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hiatal hernia, gastroes...|
+--------------------------------------------------------------------------------+-------------

In [82]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
|                                                                            text|        category|                                                             sentence_embeddings|
+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF PRESENT ILLNESS: Ms....|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gastrointestinal bleed....|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomiting. HISTORY OF PRESEN...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphagia and hematemesis w...|
+--------------------------------------------------------------------------------+-------------

In [83]:
log_folder="Mt_logs_bert"

### ClassifierDLApproach 

In [84]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(30)\
        .setLr(0.002)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

In [85]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [86]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.88      0.90        25
       Neurology       0.86      0.82      0.84        22
      Orthopedic       0.86      0.89      0.87        35
         Urology       0.90      0.95      0.93        20

        accuracy                           0.88       102
       macro avg       0.88      0.88      0.88       102
    weighted avg       0.88      0.88      0.88       102



### MultiClassifierDL 

In [87]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [88]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(8)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

In [89]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [90]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [91]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']              34
['Gastroenterology']        23
['Neurology']               21
['Urology']                 20
[]                           3
['Urology', 'Neurology']     1
Name: result, dtype: int64

In [92]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

Orthopedic          34
Gastroenterology    23
Urology             21
Neurology           21
                     3
Name: result, dtype: int64

In [93]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

                       0.00      0.00      0.00         0
Gastroenterology       0.96      0.88      0.92        25
       Neurology       0.81      0.77      0.79        22
      Orthopedic       0.88      0.86      0.87        35
         Urology       0.86      0.90      0.88        20

        accuracy                           0.85       102
       macro avg       0.70      0.68      0.69       102
    weighted avg       0.88      0.85      0.87       102



### Generic Classifier 

In [95]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

In [96]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")
      
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(8)\
    .setLearningRate(0.002)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [97]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb


In [98]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.95      0.80      0.87        25
       Neurology       0.75      0.82      0.78        22
      Orthopedic       0.93      0.74      0.83        35
         Urology       0.66      0.95      0.78        20

        accuracy                           0.81       102
       macro avg       0.82      0.83      0.81       102
    weighted avg       0.84      0.81      0.82       102



### GenericLogRegClassifier

In [99]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [100]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [101]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [102]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 25 epochs
Epoch 1/25	0.12s	Loss: 2.8436375	ACC: 0.409375
Epoch 2/25	0.01s	Loss: 2.03574	ACC: 0.60052085
Epoch 3/25	0.01s	Loss: 1.6681229	ACC: 0.7359375
Epoch 4/25	0.01s	Loss: 1.4620441	ACC: 0.7942709
Epoch 5/25	0.01s	Loss: 1.1643299	ACC: 0.8567708
Epoch 6/25	0.01s	Loss: 1.0951762	ACC: 0.8677083
Epoch 7/25	0.01s	Loss: 0.95233214	ACC: 0.8979167
Epoch 8/25	0.01s	Loss: 0.950769	ACC: 0.8958333
Epoch 9/25	0.01s	Loss: 0.85692066	ACC: 0.9151042
Epoch 10/25	0.01s	Loss: 0.78612643	ACC: 0.9203125
Epoch 11/25	0.01s	Loss: 0.7466887	ACC: 0.9296875
Epoch 12/25	0.01s	Loss: 0.8317027	ACC: 0.9078125
Epoch 13/25	0.01s	Loss: 0.70875573	ACC: 0.9385417
Epoch 14/25	0.01s	Loss: 0.68729603	ACC: 0.9322917
Epoch 15/25	0.01s	Loss: 0.7359633	ACC: 0.9125
Epoch 16/25	0.01s	Loss: 0.7532236	ACC: 0.9234375
Epoch 17/25	0.01s	Loss: 0.6184996	ACC: 0.946875
Epoch 18/25	0.01s	Loss: 0.60497105	ACC: 0.9453125
Epoch 19/25	0.01s	Loss: 0.6418084	ACC: 0.9369792
Epoch 20/25	0.02s	Loss: 0.5767652	ACC: 0.9447917
Epoch 21/25

In [103]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [104]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [105]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.88      0.92        25
       Neurology       0.77      0.77      0.77        22
      Orthopedic       0.86      0.89      0.87        35
         Urology       0.86      0.90      0.88        20

        accuracy                           0.86       102
       macro avg       0.86      0.86      0.86       102
    weighted avg       0.86      0.86      0.86       102



### GenericSVMClassifier

In [106]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [107]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")
      
gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [108]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [109]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.15s	Loss: 6.802112	ACC: 0.4484375
Epoch 2/25	0.01s	Loss: 3.7737842	ACC: 0.646875
Epoch 3/25	0.01s	Loss: 2.5159087	ACC: 0.78489584
Epoch 4/25	0.01s	Loss: 1.8442674	ACC: 0.8463542
Epoch 5/25	0.01s	Loss: 1.5127548	ACC: 0.8776042
Epoch 6/25	0.01s	Loss: 1.4778157	ACC: 0.85625
Epoch 7/25	0.01s	Loss: 1.4009507	ACC: 0.9067708
Epoch 8/25	0.01s	Loss: 1.1652503	ACC: 0.9114583
Epoch 9/25	0.01s	Loss: 1.2403356	ACC: 0.9130208
Epoch 10/25	0.01s	Loss: 1.3755908	ACC: 0.9005208
Epoch 11/25	0.01s	Loss: 0.93732697	ACC: 0.9348958
Epoch 12/25	0.01s	Loss: 1.1591876	ACC: 0.9130208
Epoch 13/25	0.01s	Loss: 1.160143	ACC: 0.9203125
Epoch 14/25	0.01s	Loss: 1.1993248	ACC: 0.8875
Epoch 15/25	0.01s	Loss: 1.0291913	ACC: 0.90625
Epoch 16/25	0.01s	Loss: 0.9239858	ACC: 0.9385417
Epoch 17/25	0.01s	Loss: 0.94648796	ACC: 0.9255208
Epoch 18/25	0.01s	Loss: 0.8318197	ACC: 0.9401042
Epoch 19/25	0.01s	Loss: 0.8413091	ACC: 0.9380208
Epoch 20/25	0.01s	Loss: 0.74829996	ACC: 0.953125
Epoch 21/25	0.01s

In [110]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [111]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [112]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.92      0.94        25
       Neurology       0.86      0.82      0.84        22
      Orthopedic       0.89      0.89      0.89        35
         Urology       0.86      0.95      0.90        20

        accuracy                           0.89       102
       macro avg       0.89      0.89      0.89       102
    weighted avg       0.89      0.89      0.89       102



## DocumentLogRegClassifier

In [113]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

logreg = DocumentLogRegClassifierApproach()\
    .setInputCols("token")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(10)\
    .setTol(1e-6)

clf_Pipeline = Pipeline(stages=[document_assembler,
                                tokenizer,
                                logreg])

doclogreg_model = clf_Pipeline.fit(trainingData)

In [114]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.88      0.90        25
       Neurology       0.80      0.73      0.76        22
      Orthopedic       0.91      0.89      0.90        35
         Urology       0.79      0.95      0.86        20

        accuracy                           0.86       102
       macro avg       0.86      0.86      0.86       102
    weighted avg       0.87      0.86      0.86       102

